In [1]:
import csv
import data_processing
import algorithm
from re import findall
import recipe_parser
import url_webscraping
import pandas as pd
import svd

In [2]:
with open('data/recipes.csv', 'r') as f:
    recipes = list(csv.DictReader(f))

In [3]:
print(recipes[0])
rah = recipe_parser.parse_instructions(recipes[0])
print(rah)

{'RecipeId': '38', 'Name': 'Low-Fat Berry Blue Frozen Dessert', 'AuthorId': '1533', 'AuthorName': 'Dancer', 'CookTime': 'PT24H', 'PrepTime': 'PT45M', 'TotalTime': 'PT24H45M', 'DatePublished': '1999-08-09T21:46:00Z', 'Description': 'Make and share this Low-Fat Berry Blue Frozen Dessert recipe from Food.com.', 'Images': 'c("https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/YUeirxMLQaeE1h3v3qnM_229%20berry%20blue%20frzn%20dess.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/AFPDDHATWzQ0b1CDpDAT_255%20berry%20blue%20frzn%20dess.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/UYgf9nwMT2SGGJCuzILO_228%20berry%20blue%20frzn%20dess.jpg", "https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,fl_progressive,q_95/v1/img/recipes/38/PeBMJN2TGSaYks2759BA_20140722_202142.jpg", \n"https://img.sndimg.com/food/image/upload/w_555,h_416,c_fit,

In [4]:
recipes = recipes[:10000]
inv_idx = data_processing.build_inverted_index(recipes)
id_to_recipe = data_processing.build_id_to_recipe(recipes)
idf = data_processing.build_idf(inv_idx, len(recipes))
recipe_norms = data_processing.build_recipe_norms(inv_idx, idf)

In [19]:
res = algorithm.cosine_similarity("lemon dessert", inv_idx, idf, recipe_norms)

In [20]:
# Print out the top 10 recipe names
for recipe_id, score in res[:10]:
    print(id_to_recipe[recipe_id]['name'], score)

Rice Pudding 0.12546715457826013
Lemon Poppy Seed Cake 0.11975219083741352
Green Tomato Pie 0.11440767944197215
Lemon Cake 0.11337507196141429
Vinegar Pie 0.1126777024986445
Lemon Sour Cream Pie 0.11215789237072495
Lemon Sponge Pie 0.1100246145192226
Orange Marmalade 0.10999133807435493
Ricotta Cheesecake 0.10658745728313504
Lemon-Sour Cream Pie 0.10578394745474305


In [21]:
res2 = algorithm.algorithm(["ice cream", "strawberry", "frozen yogurt"], inv_idx, idf, recipe_norms)

In [22]:
# Print out the top 10 recipe names
for recipe_id, score in res2[:10]:
    print(id_to_recipe[recipe_id]['name'])

Homemade Yogurt
Chocolate Cream Puffs
strawberry filling
Fresh Strawberry Pie
Frozen Yogurt-On-A-Stick
Orange Pudding Cake
Low-Fat Strawberry Butter
Strawberry Sandesh
Low-Fat Berry Blue Frozen Dessert
Xocoalt


In [2]:
recipes = pd.read_csv('data/recipes.csv')
recipes = recipes
url_webscraping.build_recipe_url_data(url_webscraping.url_builder, recipes, 'data/recipes_urls.csv')

In [8]:
urls = pd.read_csv('data/recipes_urls.csv')
urls

,RecipeId,Name,Url
0,38,Low-Fat Berry Blue Frozen Dessert,https://www.food.com/recipe/low-fat-berry-blue...
1,39,Biryani,https://www.food.com/recipe/biryani-39
2,40,Best Lemonade,https://www.food.com/recipe/best-lemonade-40
3,41,Carina's Tofu-Vegetable Kebabs,https://www.food.com/recipe/carina's-tofu-vege...
4,42,Cabbage Soup,https://www.food.com/recipe/cabbage-soup-42
...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,https://www.food.com/recipe/meg's-fresh-ginger...
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,https://www.food.com/recipe/roast-prime-rib-au...
522514,541381,Kirshwasser Ice Cream,https://www.food.com/recipe/kirshwasser-ice-cr...
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,https://www.food.com/recipe/quick-&-easy-asian...


Index(['RecipeId', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime',
       'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory',
       'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent',
       'SaturatedFatContent', 'CholesterolContent', 'SodiumContent',
       'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent',
       'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
      dtype='object')

In [7]:
documents, word_to_index, words_compressed_normed, docs_compressed_normed = svd.create_svd_docs_similarity(recipes)
similar_docs = svd.closest_recipes_to_word(documents, word_to_index, words_compressed_normed, docs_compressed_normed, 'meat', k = 5)
svd.print_closest_recipes(similar_docs)

(572, 'Aunt Kay\'s Pork Chops Casserole c("baby green peas", "potatoes", "onion", "margarine") c("Meat", "Free Of...", "< 30 Mins", "Oven") Pork', 0.7792109272776901)
(2397, 'Oven Fried Chicken II c("chicken parts", "flour", "salt", "fresh pepper", "paprika", "garlic powder", "onion powder") c("Poultry", "Meat", "High Protein", "High In...", "< 60 Mins", "Oven") Chicken', 0.7730713380921532)
(889, 'Coconut Orange Squares c("butter", "margarine", "sugar", "egg", "milk", "coconut", "flour", "baking powder", "salt") c("Dessert", "Cookie & Brownie", "Coconut", "Fruit", "Nuts", "< 15 Mins", "Oven") Bar Cookie', 0.7472714328944228)
(4187, 'Sweet Pepper Relish c("onions", "cider vinegar", "sugar", "pickling salt") c("Low Protein", "Low Cholesterol", "Healthy", "< 60 Mins", "< 30 Mins", "Canning") Vegetable', 0.7461450638603726)
(500, 'Four Treasure Chicken Soup c("water chestnuts", "chicken breast", "green onion", "soy sauce", "flour", "Chinese hot mustard", "carrot", "chicken broth", "garlic